In [36]:
# Old Model

import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sqlite3
import numpy as np

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return self.softmax(x)

conn = sqlite3.connect('../test_mTSP.sqlite3')
test_instances = pd.read_sql_query("SELECT * FROM instances", conn)
test_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)

conn.close()

test_features = test_instances[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
scaler = StandardScaler()
test_features = scaler.fit_transform(test_features)

x_test = torch.tensor(test_features, dtype=torch.float32)

# model
input_size = 12
output_size = 4
model = NeuralNetwork(input_size, output_size)
model.load_state_dict(torch.load('models/best_model.pth'))
model.eval()


with torch.no_grad():
    predictions = model(x_test)
    predicted_classes = torch.argmax(predictions, dim=1)

# mapping
with open('labels_order.txt') as f:
    label_order = [line.strip() for line in f]

rename_dict = {
    'Ant Colony': 'ant colony optimization',
    'Branch and Cut': 'branch and cut',
    'Greedy': 'greedy',
    'KMeans-Greedy': 'kmeans greedy'
}
algorithm_mapping = {i: rename_dict[label] for i, label in enumerate(label_order)}

predicted_algorithms = [algorithm_mapping[class_idx.item()] for class_idx in predicted_classes]
test_instances['predicted_algorithm'] = predicted_algorithms

# calculate composite score for each real algorithm
test_algorithms = test_algorithms.merge(
    test_instances[['instance_id', 'nr_cities']],
    on='instance_id',
    how='left'
)

test_algorithms['time_taken_norm'] = test_algorithms.groupby('instance_id')['time_taken'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
)
test_algorithms['distance_gap_norm'] = test_algorithms.groupby('instance_id')['distance_gap'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
)

is_small = test_algorithms['nr_cities'] <= 40
is_bnc = test_algorithms['strategy'].str.lower().str.contains('branch and cut')

test_algorithms['composite_score'] = (
    test_algorithms['normalized_cost'] * np.where(is_small, 0.9, 0.6) +
    test_algorithms['time_taken_norm'] * np.where(is_small & is_bnc, 0, np.where(is_small, 0.1, 0.2)) +
    test_algorithms['distance_gap_norm'] * 0.05
)

#  best real algorithm for each instance
best_real = test_algorithms.loc[test_algorithms.groupby('instance_id')['composite_score'].idxmin()]
best_real = best_real[['instance_id', 'strategy']].rename(columns={'strategy': 'real_best_algorithm'})

# compare predictions with real best algorithms
comparison = test_instances[['instance_id', 'predicted_algorithm']].merge(best_real, on='instance_id')
reverse_rename = {
    'ant colony optimization': 'Ant Colony',
    'branch and cut': 'Branch and Cut',
    'greedy': 'Greedy',
    'kmeans greedy': 'KMeans-Greedy'
}

comparison['predicted_algorithm_match'] = comparison['predicted_algorithm'].map(reverse_rename)
comparison['match'] = (
    comparison['predicted_algorithm_match'] == comparison['real_best_algorithm']
)
comparison.to_csv('comparatie_pred_vs_real.csv', index=False)

accuracy = comparison['match'].mean()
print(f"Acuratetea retelei neruonale vechi: {accuracy:.2%}")
print(comparison.head())

Acuratetea retelei neruonale vechi: 68.32%
   instance_id predicted_algorithm real_best_algorithm  \
0            1      branch and cut      Branch and Cut   
1            2      branch and cut      Branch and Cut   
2            3      branch and cut      Branch and Cut   
3            4      branch and cut      Branch and Cut   
4            5      branch and cut      Branch and Cut   

  predicted_algorithm_match  match  
0            Branch and Cut   True  
1            Branch and Cut   True  
2            Branch and Cut   True  
3            Branch and Cut   True  
4            Branch and Cut   True  


In [37]:
# New Model

import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sqlite3
import numpy as np

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

conn = sqlite3.connect('../test_mTSP.sqlite3')
test_instances = pd.read_sql_query("SELECT * FROM instances", conn)
test_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)

conn.close()

test_features = test_instances[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
scaler = StandardScaler()
test_features = scaler.fit_transform(test_features)

x_test = torch.tensor(test_features, dtype=torch.float32)

# model
input_size = 12
output_size = 4
model = NeuralNetwork(input_size, output_size)
model.load_state_dict(torch.load('models_version2/best_model.pth'))
model.eval()


with torch.no_grad():
    predictions = model(x_test)
    predicted_classes = torch.argmax(predictions, dim=1)

# mapping
with open('labels_order.txt') as f:
    label_order = [line.strip() for line in f]

rename_dict = {
    'Ant Colony': 'ant colony optimization',
    'Branch and Cut': 'branch and cut',
    'Greedy': 'greedy',
    'KMeans-Greedy': 'kmeans greedy'
}
algorithm_mapping = {i: rename_dict[label] for i, label in enumerate(label_order)}

predicted_algorithms = [algorithm_mapping[class_idx.item()] for class_idx in predicted_classes]
test_instances['predicted_algorithm'] = predicted_algorithms

# calculate composite score for each real algorithm
test_algorithms = test_algorithms.merge(
    test_instances[['instance_id', 'nr_cities']],
    on='instance_id',
    how='left'
)

test_algorithms['time_taken_norm'] = test_algorithms.groupby('instance_id')['time_taken'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
)
test_algorithms['distance_gap_norm'] = test_algorithms.groupby('instance_id')['distance_gap'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
)

is_small = test_algorithms['nr_cities'] <= 40
is_bnc = test_algorithms['strategy'].str.lower().str.contains('branch and cut')

test_algorithms['composite_score'] = (
    test_algorithms['normalized_cost'] * np.where(is_small, 0.9, 0.6) +
    test_algorithms['time_taken_norm'] * np.where(is_small & is_bnc, 0, np.where(is_small, 0.1, 0.2)) +
    test_algorithms['distance_gap_norm'] * 0.05
)

#  best real algorithm for each instance
best_real = test_algorithms.loc[test_algorithms.groupby('instance_id')['composite_score'].idxmin()]
best_real = best_real[['instance_id', 'strategy']].rename(columns={'strategy': 'real_best_algorithm'})


comparison = test_instances[['instance_id', 'predicted_algorithm']].merge(best_real, on='instance_id')
reverse_rename = {
    'ant colony optimization': 'Ant Colony',
    'branch and cut': 'Branch and Cut',
    'greedy': 'Greedy',
    'kmeans greedy': 'KMeans-Greedy'
}

comparison['predicted_algorithm_match'] = comparison['predicted_algorithm'].map(reverse_rename)
comparison['match'] = (
    comparison['predicted_algorithm_match'] == comparison['real_best_algorithm']
)

comparison.to_csv('comparatie_pred_vs_real_version2.csv', index=False)

accuracy = comparison['match'].mean()
print(f"Acuratetea retelei neruonale vechi: {accuracy:.2%}")
print(comparison.head())

Acuratetea retelei neruonale vechi: 55.45%
   instance_id predicted_algorithm real_best_algorithm  \
0            1      branch and cut      Branch and Cut   
1            2      branch and cut      Branch and Cut   
2            3      branch and cut      Branch and Cut   
3            4      branch and cut      Branch and Cut   
4            5      branch and cut      Branch and Cut   

  predicted_algorithm_match  match  
0            Branch and Cut   True  
1            Branch and Cut   True  
2            Branch and Cut   True  
3            Branch and Cut   True  
4            Branch and Cut   True  
